In [57]:
import pandas as pd

outliers = pd.DataFrame()
data = pd.read_csv('horse_data.csv', na_values='?')
data.columns = ['операция?', 'Возраст', 'Номер больницы', 'Ректальная температура', 'Пульс', 'Частота дыхания',
              'Температура конечностей', 'Периферический пульс', 'Слизистые оболочки', 'Время наполнения капилляров',
              'Боль', 'Перистальтика', 'Вздутие живота', 'Назогастральный зонд', 'Назогастральный рефлюкс',
              'Назогастральный рефлюкс РН', 'Ректальное исследование', 'Живот', 'Объём упакованной ячейки', 'Общий белок',
              'Внешний вид брюшной полости', 'Общий белок брюшной полости', 'Результат', 'Хирургическое поражение?',
              'тип поражения_1', 'тип поражения_2', 'тип поражения_3', 'Паталогии']
numeric_columns = ['Ректальная температура', 'Пульс', 'Частота дыхания', 'Назогастральный рефлюкс РН',
                  'Объём упакованной ячейки', 'Общий белок', 'Общий белок брюшной полости']

data_with_num_columns = data.reindex(numeric_columns, axis=1)
for column_name in numeric_columns:
    Q1 = data_with_num_columns[column_name].quantile(0.2)
    Q3 = data_with_num_columns[column_name].quantile(0.8)
    IQR = Q3 - Q1
    for outlier in data_with_num_columns[column_name]: 
        if outlier < (Q1 - (1.5 * IQR)):
            row = {'Параметр': column_name, 'минимальный выброс': outlier, 
                   'максимальный выброс': 'NaN', 'значение квантиля': Q1}
            outliers = pd.concat([outliers, pd.DataFrame([row])])
        elif outlier > (Q3 + (1.5 * IQR)):
            row = {'Параметр': column_name, 'минимальный выброс': 'NaN', 
                   'максимальный выброс': outlier, 'значение квантиля': Q3}
            outliers = pd.concat([outliers, pd.DataFrame([row])])
        else:
            None
outliers.reset_index().drop('index', 1)

,Параметр,минимальный выброс,максимальный выброс,значение квантиля
0,Ректальная температура,35.4,NaN,37.60
1,Ректальная температура,NaN,40.3,38.60
2,Ректальная температура,NaN,40.3,38.60
3,Ректальная температура,36,NaN,37.60
4,Ректальная температура,NaN,40.8,38.60
5,Пульс,NaN,184,96.00
6,Частота дыхания,NaN,84,40.00
7,Частота дыхания,NaN,96,40.00
8,Частота дыхания,NaN,80,40.00
9,Частота дыхания,NaN,80,40.00


* Ректальная температура - скорее всего выбросами не являются, при тяжелом заболевании, либо позднем шоке возможны такие значения, следует учесть при расчётах.
* Пульс - скорее всего выброс, т.к. значение превышает в два раза значение 80 процентиля, возможно у лошади было какое то стрессовое состояние при проведении замеров, не следует учитывать при расчётах, за счёт данного значения будет происходить искажение всех данных.
* Частота дыхания - скорее всего выбросы, превышение 80 процентиля в два раза, возможно замеры производились некорректно (например после тренировки лошади), не следует учитывать в расчётах.
* Общий белок брюшной полости - скорее всего выбросы, превышение 80 процентиля в два раза, возможно замеры были сделаны во время какого то воспалительного процесса у лошади, не следует учитывать в расчётах.